Saaksshi Jilhewar - sjilhewa \
Gauri Dalwankar - gdalwank

In [1]:
#ignore warnings

import warnings
warnings.filterwarnings('ignore')


In [2]:
#importing libraries

import findspark
import pyspark
import pyspark.sql.functions as F
import pyspark
from pyspark.sql import SparkSession, SQLContext
from pyspark.ml import Pipeline,Transformer
from pyspark.ml.feature import Imputer,StandardScaler,StringIndexer,OneHotEncoder, VectorAssembler

from pyspark.sql.functions import *
from pyspark.sql.types import *
import numpy as np

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix

import itertools

In [3]:
#creating spark session

findspark.init()
findspark.find()

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("MQTT") \
    .getOrCreate()

spark = SparkSession.builder.appName("mqttProject").getOrCreate()
sc    = spark.sparkContext

sqlContext = SQLContext(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/11/28 15:58:08 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
22/11/28 15:58:08 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
22/11/28 15:58:08 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
22/11/28 15:58:08 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [4]:
#importing dataset

test = spark.read.csv( ("gs://dataproc-staging-us-west1-657840282044-odudoeds/test30_augmented.csv"),header=True, inferSchema= True)
train = spark.read.csv( ("gs://dataproc-staging-us-west1-657840282044-odudoeds/train70_augmented.csv"),header=True, inferSchema= True)

In [5]:
#defining nominal and continuous columns, we figured these columns while doing Data Engineering part

col_names = ['tcp_flags',
 'tcp_time_delta',
 'tcp_len',
 'mqtt_conack_flags',
 'mqtt_conack_flags_reserved',
 'mqtt_conack_flags_sp',
 'mqtt_conack_val',
 'mqtt_conflag_cleansess',
 'mqtt_conflag_passwd',
 'mqtt_conflag_qos',
 'mqtt_conflag_reserved',
 'mqtt_conflag_retain',
 'mqtt_conflag_uname',
 'mqtt_conflag_willflag',
 'mqtt_conflags',
 'mqtt_dupflag',
 'mqtt_hdrflags',
 'mqtt_kalive',
 'mqtt_len',
 'mqtt_msg',
 'mqtt_msgid',
 'mqtt_msgtype',
 'mqtt_proto_len',
 'mqtt_qos',
 'mqtt_retain',
 'mqtt_sub_qos',
 'mqtt_suback_qos',
 'mqtt_ver',
 'mqtt_willmsg',
 'mqtt_willmsg_len',
 'mqtt_willtopic',
 'mqtt_willtopic_len',
 'target']

nominal_cols = []

continuous_cols = ['tcp_time_delta',
 'tcp_len',
 'mqtt_conack_flags_reserved',
 'mqtt_conack_flags_sp',
 'mqtt_conack_val',
 'mqtt_conflag_cleansess',
 'mqtt_conflag_passwd',
 'mqtt_conflag_qos',
 'mqtt_conflag_reserved',
 'mqtt_conflag_retain',
 'mqtt_conflag_uname',
 'mqtt_conflag_willflag',
 'mqtt_dupflag',
 'mqtt_kalive',
 'mqtt_len',
 'mqtt_msgid',
 'mqtt_msgtype',
 'mqtt_proto_len',
 'mqtt_qos',
 'mqtt_retain',
 'mqtt_sub_qos',
 'mqtt_suback_qos',
 'mqtt_ver',
 'mqtt_willmsg',
 'mqtt_willmsg_len',
 'mqtt_willtopic',
 'mqtt_willtopic_len']

In [6]:
#renaming columns 

train = train.toDF(*(c.replace('.', '_') for c in train.columns))
test = test.toDF(*(c.replace('.', '_') for c in test.columns))

In [7]:
#dropping columns
train = train.drop('tcp_flags',
 'mqtt_conack_flags',
 'mqtt_conflags',
 'mqtt_hdrflags',
 'mqtt_msg',
 'mqtt_protoname')
test = test.drop('tcp_flags',
 'mqtt_conack_flags',
 'mqtt_conflags',
 'mqtt_hdrflags',
 'mqtt_msg',
 'mqtt_protoname')

In [8]:
#defining columns to drop, which were to high in correlation this we figured out while doing Data Engineering
#dropping nominal columns because it has string values, and we kept getting string indexer error

to_drop =  ["mqtt_conflag_cleansess","mqtt_proto_len","mqtt_conflag_passwd","mqtt_qos"]

When we have to figure out if it is a attack or not, the problem becomes binary classification, i.e. it either 1, when it's an attack or 0, which is a case when it's not an attack(legitimate) in our data set case.

But, when we have to predict what kind of attack it is we need to do multiclass classification.
In our code we have assigned these values explicitly. \
0.0 -> 'legitimate' \
1.0 -> "flood" \
2.0 -> "dos" \
3.0 -> "bruteforce" \
4.0 -> "slowite" \
5.0 -> "malformed"


In [9]:
class OutcomeCreater_binary(Transformer): # this defines a transformer that creates the outcome column which tell if it's an attack or not
    
    def __init__(self):
        super().__init__()
  
    def _transform(self, dataset):
        label_to_binary = udf(lambda name: 0.0 if name == 'legitimate' else 1.0)
        output_df = dataset.withColumn('outcome', label_to_binary(col('target'))).drop("target")  
        output_df = output_df.withColumn('outcome', col('outcome').cast(DoubleType()))
        return output_df
    
class OutcomeCreater_multi(Transformer): # this defines a transformer that creates the outcome column ehich defines the type of attack
    
    def __init__(self):
        super().__init__()
  
    def _transform(self, dataset):
        label_to_multiple = udf(lambda name: 0.0 if name == 'legitimate' else (1.0 if name == "flood" else(2.0 if name == "dos" else(3.0 if name == "bruteforce" else(4.0 if name == "slowite" else (5.0))))))
        output_df = dataset.withColumn('outcome', label_to_multiple(col('target'))).drop("target")  
        output_df = output_df.withColumn('outcome', col('outcome').cast(DoubleType()))
        return output_df
        
class FeatureTypeCaster(Transformer): # this transformer will cast the columns as appropriate types  
    def __init__(self):
        super().__init__()

    def _transform(self, dataset):
        output_df = dataset
        for col_name in continuous_cols:
            output_df = output_df.withColumn(col_name,col(col_name).cast(DoubleType()))

        return output_df
    
class ColumnDropper(Transformer): # this transformer drops unnecessary columns
    def __init__(self, columns_to_drop = None):
        super().__init__()
        self.columns_to_drop=columns_to_drop
    def _transform(self, dataset):
        output_df = dataset
        for col_name in self.columns_to_drop:
            output_df = output_df.drop(col_name)
            
        return output_df
    
def get_preprocess_pipeline(classification):
    # Stage where columns are casted as appropriate types
    stage_typecaster = FeatureTypeCaster()

    # Stage where nominal columns are transformed to index columns using StringIndexer
    nominal_id_cols = [x+"_index" for x in nominal_cols]
    nominal_onehot_cols = [x+"_encoded" for x in nominal_cols]
    stage_nominal_indexer = StringIndexer(inputCols = nominal_cols, outputCols = nominal_id_cols )
    
    # Stage where the index columns are further transformed using OneHotEncoder
    stage_nominal_onehot_encoder = OneHotEncoder(inputCols=nominal_id_cols, outputCols=nominal_onehot_cols)

    # Stage where all relevant features are assembled into a vector (and dropping a few)
    feature_cols = continuous_cols+nominal_onehot_cols
    corelated_cols_to_remove = to_drop
    
    for col_name in corelated_cols_to_remove:
        feature_cols.remove(col_name)
    stage_vector_assembler = VectorAssembler(inputCols=feature_cols, outputCol="vectorized_features")

    # Stage where we scale the columns
    stage_scaler = StandardScaler(inputCol= 'vectorized_features', outputCol= 'features')
    

    # Stage for creating the outcome column representing whether there is attack/type of attack
    if(classification == "binary"): stage_outcome = OutcomeCreater_binary()
    else: stage_outcome = OutcomeCreater_multi()

    # Removing all unnecessary columns, only keeping the 'features' and 'outcome' columns
    stage_column_dropper = ColumnDropper(columns_to_drop = nominal_cols+nominal_id_cols+
        nominal_onehot_cols + continuous_cols + ['vectorized_features'])
    
    pipeline = Pipeline(stages=[stage_typecaster,stage_nominal_indexer,stage_nominal_onehot_encoder,
        stage_vector_assembler,stage_scaler,stage_outcome,stage_column_dropper])
    
    return pipeline 

In [10]:
preprocess_pipeline = get_preprocess_pipeline("binary")
preprocess_pipeline_model = preprocess_pipeline.fit(train)

train_df_binary = preprocess_pipeline_model.transform(train)
test_df_binary = preprocess_pipeline_model.transform(test)

22/11/28 15:59:37 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [11]:
preprocess_pipeline = get_preprocess_pipeline("multi")
preprocess_pipeline_model = preprocess_pipeline.fit(train)

train_df_multi = preprocess_pipeline_model.transform(train)
test_df_multi = preprocess_pipeline_model.transform(test)

In [12]:
train_df_binary.printSchema()
test_df_binary.printSchema()

root
 |-- features: vector (nullable = true)
 |-- outcome: double (nullable = true)

root
 |-- features: vector (nullable = true)
 |-- outcome: double (nullable = true)



In [13]:
train_df_multi.printSchema()
test_df_multi.printSchema()

root
 |-- features: vector (nullable = true)
 |-- outcome: double (nullable = true)

root
 |-- features: vector (nullable = true)
 |-- outcome: double (nullable = true)



In [14]:
train_df_binary.show(10)

+--------------------+-------+
|            features|outcome|
+--------------------+-------+
|(23,[0,1,12,14],[...|    0.0|
|(23,[0],[3.661522...|    1.0|
|(23,[0],[1.464608...|    1.0|
|(23,[0],[1.464608...|    1.0|
|(23,[0,1,12,14],[...|    1.0|
|(23,[0,1,12,14],[...|    0.0|
|(23,[0,1,12,14],[...|    0.0|
|(23,[0],[2.013837...|    1.0|
|(23,[0,1,12,14],[...|    0.0|
|(23,[0,1,8,11,12,...|    1.0|
+--------------------+-------+
only showing top 10 rows



In [15]:
train_df_multi.show(10)

+--------------------+-------+
|            features|outcome|
+--------------------+-------+
|(23,[0,1,12,14],[...|    0.0|
|(23,[0],[3.661522...|    3.0|
|(23,[0],[1.464608...|    1.0|
|(23,[0],[1.464608...|    4.0|
|(23,[0,1,12,14],[...|    1.0|
|(23,[0,1,12,14],[...|    0.0|
|(23,[0,1,12,14],[...|    0.0|
|(23,[0],[2.013837...|    3.0|
|(23,[0,1,12,14],[...|    0.0|
|(23,[0,1,8,11,12,...|    3.0|
+--------------------+-------+
only showing top 10 rows



In [16]:
# # limiting the dataset, becase we kept getting WIN100061 error as before while doing Task 1
train_df_binary = train_df_binary.limit(100000)
test_df_binary = test_df_binary.limit(10000)

In [17]:
# # limiting the dataset, becase we kept getting error as before while doing Task 1
train_df_multi = train_df_multi.limit(100000)
test_df_multi = test_df_multi.limit(10000)

## Predicting type of attack

### Decision Tree

In [18]:
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier(featuresCol = "features", labelCol = "outcome")
dt_model = dt.fit(train_df_multi)

Traceback (most recent call last):                                 (0 + 4) / 17]
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError
Traceback (most recent call last):                                 (1 + 4) / 17]
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_in

In [19]:
dt_prediction_train = dt_model.transform(train_df_multi)
dt_prediction_test = dt_model.transform(test_df_multi)

dt_accuracy_train = (dt_prediction_train.filter(
    dt_prediction_train.outcome == dt_prediction_train.prediction).count() / 
    float(dt_prediction_train.count()))
dt_accuracy_test = (dt_prediction_test.filter(
    dt_prediction_test.outcome == dt_prediction_test.prediction).count()
    / float(dt_prediction_test.count()))

print(f"Train accuracy = {np.round(dt_accuracy_train*100,2)}%")
print(f"Test accuracy = {np.round(dt_accuracy_test*100,2)}%")

Traceback (most recent call last):                                 (0 + 4) / 17]
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError
Traceback (most recent call last):                                 (1 + 4) / 17]
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_in

Train accuracy = 72.56%
Test accuracy = 72.53%


##### Hyper parameter tuning and cross validation

In [44]:
# Create ParamGrid for Cross Validation
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

dtparamGrid = (ParamGridBuilder()
             .addGrid(dt.maxDepth, [10, 20])
             .addGrid(dt.maxBins, [20, 30])
             .build())

evaluator = MulticlassClassificationEvaluator(labelCol='outcome', predictionCol="prediction", metricName="accuracy")

dt_cv = CrossValidator(estimator=dt, estimatorParamMaps=dtparamGrid, 
                    evaluator=evaluator, numFolds=2)

dt_cv_model = dt_cv.fit(train_df_multi)
dt_cv_prediction_test = dt_cv_model.transform(test_df_multi)

Traceback (most recent call last):                                 (0 + 4) / 17]
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError
Traceback (most 

In [45]:
print(f'Test Accuracy after Cross-Validation in {(evaluator.evaluate(dt_cv_prediction_test)*100)}%')

Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError
Traceback (most recent call last):
  File "/usr/lib/spark/pyth

Test Accuracy after Cross-Validation in 80.58%


Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError
Traceback (most recent call last):
  File "/usr/lib/spark/pyth

### Logistic Regression

In [50]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol = 'features', labelCol = 'outcome')
lrModel = lr.fit(train_df_multi)

Traceback (most recent call last):                                 (0 + 4) / 17]
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError
Traceback (most 

In [51]:
predictions_train = lrModel.transform(train_df_multi)
predictions_test = lrModel.transform(test_df_multi)

train_accuracy = evaluator.evaluate(predictions_train)
test_accuracy = evaluator.evaluate(predictions_test)

print(f"Train accuracy = {np.round(train_accuracy*100,2)}%")
print(f"Test accuracy = {np.round(test_accuracy*100,2)}%")

Traceback (most recent call last):                                 (0 + 4) / 17]
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError
Traceback (most 

Train accuracy = 65.69%
Test accuracy = 65.16%


Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError


##### Hyper parameter tuning and cross validation

In [52]:
# Create ParamGrid for Cross Validation
lr_paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.05,  1, 0.005, 2, 0.5])
             .addGrid(lr.maxIter, [100, 50, 70, 200, 5])
             .build())

lr_cv = CrossValidator(estimator=lr, estimatorParamMaps=lr_paramGrid, 
                    evaluator=evaluator, numFolds=2)

lr_cv_model = lr_cv.fit(train_df_multi)
lr_cv_prediction_test = lr_cv_model.transform(test_df_multi)

Traceback (most recent call last):                                 (0 + 4) / 17]
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError
Traceback (most recent call last):                                 (1 + 4) / 17]
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_in

In [53]:
print('Test Accuracy after Cross-Validation in %:', (evaluator.evaluate(lr_cv_prediction_test)*100))

Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError
Traceback (most recent call last):==================>         

Test Accuracy after Cross-Validation in %: 64.03999999999999


## Predicting if attack

### Decision Tree

In [26]:
dt_model = dt.fit(train_df_binary)

evaluator = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction', 
    labelCol='outcome', metricName='areaUnderROC')

Traceback (most recent call last):                                 (0 + 4) / 17]
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError
Traceback (most recent call last):                                 (1 + 4) / 17]
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_in

In [27]:
dt_prediction_train = dt_model.transform(train_df_binary)
dt_prediction_test = dt_model.transform(test_df_binary)

dt_accuracy_train = (dt_prediction_train.filter(
    dt_prediction_train.outcome == dt_prediction_train.prediction).count() / 
    float(dt_prediction_train.count()))
dt_accuracy_test = (dt_prediction_test.filter(
    dt_prediction_test.outcome == dt_prediction_test.prediction).count()
    / float(dt_prediction_test.count()))

print(f"Train accuracy = {np.round(dt_accuracy_train*100,2)}%")
print(f"Test accuracy = {np.round(dt_accuracy_test*100,2)}%")

Traceback (most recent call last):                                 (0 + 4) / 17]
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError
Traceback (most recent call last):                                 (1 + 4) / 17]
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_in

Train accuracy = 90.83%
Test accuracy = 90.28%


##### Hyper parameter tuning and cross validation

In [28]:
# Create ParamGrid for Cross Validation
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

dtparamGrid = (ParamGridBuilder()
             .addGrid(dt.maxDepth, [5, 10])
             .addGrid(dt.maxBins, [10, 20])
             .build())

dt_cv = CrossValidator(estimator=dt, estimatorParamMaps=dtparamGrid, 
                    evaluator=evaluator, numFolds=2)

dt_cv_model = dt_cv.fit(train_df_binary)
dt_cv_prediction_test = dt_cv_model.transform(test_df_binary)

Traceback (most recent call last):                                 (1 + 4) / 17]
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError
Traceback (most 

In [29]:
print(f'Test AUC after Cross-Validation and parameter tuning is {(evaluator.evaluate(dt_cv_prediction_test)*100)}%')


Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError
Traceback (most recent call last):==========>                 

Test AUC after Cross-Validation and parameter tuning is 90.46053984610806%


### Logistic Regression

In [30]:
lrModel = lr.fit(train_df_binary)

Traceback (most recent call last):                                 (0 + 4) / 17]
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError
Traceback (most 

In [31]:
predictions_train = lrModel.transform(train_df_binary)
predictions_test = lrModel.transform(test_df_binary)

train_accuracy = evaluator.evaluate(predictions_train)
test_accuracy = evaluator.evaluate(predictions_test)

print(f"Train AUC{np.round(train_accuracy*100,2)}%")
print(f"Test AUC = {np.round(test_accuracy*100,2)}%")

Traceback (most recent call last):                                 (0 + 4) / 17]
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError
Traceback (most 

Train accuracy = 80.61%
Test accuracy = 82.4%


In [33]:
lr_paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.05,  1])
             .addGrid(lr.maxIter, [10, 15])
             .build())

lr_cv = CrossValidator(estimator=lr, estimatorParamMaps=lr_paramGrid, 
                    evaluator=evaluator, numFolds=2)

lr_cv_model = lr_cv.fit(train_df_binary)
lr_cv_prediction_test = lr_cv_model.transform(test_df_binary)

Traceback (most recent call last):                                 (0 + 4) / 17]
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError
Traceback (most 

In [34]:
print(f'Test AUC after Cross-Validation and parameter tuning is {(evaluator.evaluate(lr_cv_prediction_test)*100)}%')


Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError
Traceback (most recent call last):
  File "/usr/lib/spark/pyth

Test AUC after Cross-Validation and parameter tuning is 79.29355420507632%
